In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import time
import re

In [ ]:
headers={'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}

In [ ]:
# Resturants urls
links=pd.read_csv("links.csv")
links.head()

In [ ]:
url="http://swiggy.com"

In [ ]:
url_rest=links['links'].values

In [ ]:
urls = url + links['links'].astype(str)
urls=urls.values

In [ ]:
# Pareser to get data of each restaurants
def parse(url):
    try:
        response=requests.get(url,headers=headers)
        time.sleep(5)
        if response.status_code==200:
            content=response.content
            soup=BeautifulSoup(content,"html.parser")
            data=json.loads(soup.find('script', type='application/ld+json').text)
            nam=data['name']
            reg=data['address']['addressRegion']
            lon=data['geo']['longitude']
            lat=data['geo']['latitude']
            pri=data['priceRange']
            rat=data['aggregateRating']['ratingValue']
    except Exception as ex:
        print(str(ex))
    return nam,lon,lat,pri,reg,rat

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures

In [ ]:
# Multiprocessing to speed up scraping
with ProcessPoolExecutor(max_workers=4) as executor:
    future_results = {executor.submit(parse, url): url for url in urls}
    results = []
    for future in concurrent.futures.as_completed(future_results):
        results.append(future.result())

In [ ]:
df=pd.DataFrame(list(results),columns = ["Restaurant Name","Longitude","Latitude","Price", "Region","Rating"])

In [ ]:
df.head()

In [ ]:
# Parsing price for two
price=df["Price"]
price_for_two=[]
for i in range(len(price)):
    en=price[i].encode('ascii','ignore')
    en=en.decode('utf-8') 
    pr=re.findall(r'\d+',en)
    for i in pr:  
        price_for_two.append(i)

In [ ]:
df["Price For Two"]=price_for_two

In [ ]:
df=df.drop('Price',axis=1)
df.head()

In [ ]:
# Final Data
df.to_csv("scrape_data.csv",index=False)